### Homework: Vector Search

In this homework, we will implement an end-to-end semantic search engine.

### Q1. Prepare Documents

Import documents.json, read the file and prepare the dataset:

```
import json

with open('documents.json', 'rt') as f_in:
  docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
   for doc in course_dict['documents']:
      doc['course'] = course_dict['course']
      documents.append(doc)

len(documents)
```

How many records do we have in the pre-processed "documents"?

* 1000
* 1051
* 901
* 948

In [13]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

### A1. 948

### Q2. Create Embeddings Using Pretrained Models

Import the sentence transformer library.  Please review the Sentence Transformer pretrained documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview

This is a new library compared to the previous modules. 
Please perform "pip install sentence_transformers==2.7.0"

```
from sentence_transformers import SentenceTransformer
```

If you get an error do the following:
* 1. Uninstall numpy 
*  2. Uninstall torch
* 3. pip install numpy==1.26.4
* 4. pip install torch
* run the above cell, it should work

```
model = SentenceTransformer("all-MiniLM-L12-v2")
```

https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2

What is the model size (in MB) and average performance?

* [420, 63.30]
* [120, 59.76]
* [290, 59.84]
* [420, 51.72]

In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
model = SentenceTransformer("all-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### A2. The model size is 120 MB and the average performance is 59.76.

### Q3. Get the dimension for model embedding

```
len(model.encode("This is a simple sentence"))
```

What is the dimension of the model embedding?

* 768
* 265
* 384
* 1056

In [16]:
len(model.encode("This is a simple sentence"))

384

### A3. 384

```
# created dense vector using the pre-trained model
operations = []
for doc in documents:
   # Transforming the title into an embedding using the model
   doc["question_vector"] = model.encode(doc["question"]).tolist()
   operations.append(doc)
```

In [17]:
# created dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

Establish a connection to Elasticsearch

```
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

es_client.info()
``` 

In [18]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '24fa0a14c17b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'IkbJ06R7QAOrlv5sEmIRxw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Q4. Create Mappings and Index

In the mappings, change "section" to "keyword" type.

```
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)
```

In [19]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Add documents into the index.

```
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

```

In [20]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In the es_client.indices.delete statement, what does "ignore_unavailable" mean?

* If the pre-trained model is unavailable, skip this statement
* If the index is unavailable, skip this statement
* If the pre-trained model is unavailable, don't skip this statement
* If the index is unavailable, don't skip this statement

### A4. If the index is unavailable, skip this statement.

https://elasticsearch-py.readthedocs.io/en/7.x/api.html#indices
```
 delete(index, params=None, headers=None)

    Deletes an index.

    https://www.elastic.co/guide/en/elasticsearch/reference/7.x/indices-delete-index.html
    Parameters:	

        index – A comma-separated list of indices to delete; use _all or * string to delete all indices
        allow_no_indices – Ignore if a wildcard expression resolves to no concrete indices (default: false)
        expand_wildcards – Whether wildcard expressions should get expanded to open, closed, or hidden indices Valid choices: open, closed, hidden, none, all Default: open,closed
        ignore_unavailable – Ignore unavailable indexes (default: false)
        master_timeout – Specify timeout for connection to master
        timeout – Explicit operation timeout
```

### Q5. Create End User Query and Perform Semantic Search

Use the search term "how to enrol to course?" and perform a semantic search

```
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]
```

What is the similarity score, section and course for the first result?


* [0.78, 'Module 1: Introduction', 'mlops-zoomcamp']
* [0.74, 'General course-related questions', 'data-engineering-zoomcamp']
* [0.72, 'Projects (Midterm and Capstone)', 'machine-learning-zoomcamp']
* [0.71, 'General course-related questions', 'machine-learning-zoomcamp']

In [21]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'YaWYf5AB1U3PcFinZ3Pd',
  '_score': 0.852048,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}},
 {'_index': 'course-questions',
  '_id': 'gqWYf5AB1U3PcFina3Nu',
  '_score': 0.8381671,
  '_source': {'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'}},
 {'_index': 'course-questions',
  '_id': 'JKWYf5AB1U3PcFinlHWf',
  '_score': 0.8284229,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'Gene

### A5. The closest answer is [0.74, 'General course-related questions', 'data-engineering-zoomcamp']

https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-knn-query.html


### Q6. Perform Semantic Search & Filtering

Filter the results to "General course-related questions" section only:

```
knn_query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
            },
        },
        
    knn=knn_query,
    size=5
)

response["hits"]["hits"]
```

Do you see the results filtered only to the "General course-related questions" section?

* Yes
* No

In [26]:
knn_query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
            "section": "General course-related questions"
        },
    },
        
    knn=knn_query,
    size=5
)

response["hits"]["hits"]

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: field [text_vector] does not exist in the mapping')